In [ ]:
from fenics import *
import mshr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Physical Parameters
k = 200         # Thermal conductivity (W/m·K)
h = 10          # Heat transfer coefficient (W/m²·K)
T_tube = 100    # Tube temperature (°C)
T_amb = 25      # Ambient temperature (°C)

# Geometry Parameters
w_fin = 0.1     # Fin width (m)
H_fin = 0.2     # Fin height (m)
R_tube = 0.02   # Tube radius (m)

# Create the rectangular domain
rectangle = mshr.Rectangle(Point(0, 0), Point(w_fin, H_fin))

# Create the circular hole (tube)
circle = mshr.Circle(Point(w_fin / 2, H_fin / 2), R_tube)

# Create the domain by subtracting the circle from the rectangle
domain = rectangle - circle

# Generate the mesh
mesh_resolution = 64  # Adjust for finer mesh
mesh = mshr.generate_mesh(domain, mesh_resolution)

# Define the function space
V = FunctionSpace(mesh, 'P', 1)

# Define boundaries using SubDomain
class TubeBoundary(SubDomain):
    def inside(self, x, on_boundary):
        center = np.array([w_fin / 2, H_fin / 2])
        r = np.sqrt((x[0] - center[0])**2 + (x[1] - center[1])**2)
        return near(r, R_tube, DOLFIN_EPS) and on_boundary

class TopBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], H_fin, DOLFIN_EPS) and on_boundary

class SideBottomBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return (near(x[0], 0, DOLFIN_EPS) or near(x[0], w_fin, DOLFIN_EPS) or near(x[1], 0, DOLFIN_EPS)) and on_boundary

# Initialize boundary conditions
tube_boundary = TubeBoundary()
top_boundary = TopBoundary()
side_bottom_boundary = SideBottomBoundary()

# Mark boundaries
boundary_markers = MeshFunction('size_t', mesh, mesh.topology().dim() - 1, 0)
tube_boundary.mark(boundary_markers, 1)
top_boundary.mark(boundary_markers, 2)
side_bottom_boundary.mark(boundary_markers, 3)

# Define measure for boundary integrals
ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

# Define the variational problem
T = Function(V)           # Solution function
v = TestFunction(V)       # Test function

# Material properties
kappa = Constant(k)
h_coeff = Constant(h)
T_ambient = Constant(T_amb)

# Weak form
a = kappa * dot(grad(T), grad(v)) * dx + h_coeff * (T - T_ambient) * v * dx + h_coeff * (T - T_ambient) * v * ds(2)
L = Constant(0) * v * dx  # Right-hand side is zero

# Dirichlet boundary condition on the tube surface
bc_tube = DirichletBC(V, Constant(T_tube), tube_boundary)
bcs = [bc_tube]

# Solve the problem
solve(a == L, T, bcs)

# Compute the normal heat flux at the tube boundary
n = FacetNormal(mesh)
flux = -kappa * dot(grad(T), n)

# Integrate the heat flux over the tube boundary (Marker 1)
total_heat_flux_tube = assemble(flux * ds(1))

print(f"Total heat transfer at the tube interface: {total_heat_flux_tube:.2f} W per unit depth")

# Compute total convective heat loss over the fin domain
q_conv_domain = h_coeff * (T - T_ambient)
total_heat_loss_domain = assemble(q_conv_domain * dx)

# Compute convective heat loss over the top boundary (Marker 2)
q_conv_top = h_coeff * (T - T_ambient)
total_heat_loss_top = assemble(q_conv_top * ds(2))

total_heat_loss = total_heat_loss_domain + total_heat_loss_top

print(f"Total heat loss over the fin due to convection: {total_heat_loss:.2f} W per unit depth")

# Plot the temperature distribution
T_array = T.compute_vertex_values(mesh)
coords = mesh.coordinates()

plt.figure(figsize=(8, 6))
plt.tripcolor(coords[:, 0], coords[:, 1], mesh.cells(), T_array, shading='flat', cmap='hot')
plt.colorbar(label='Temperature (°C)')
plt.title('Temperature Distribution in the Fin')
plt.xlabel('Width (m)')
plt.ylabel('Height (m)')
plt.axis('equal')
plt.show()